# Project: Wheel of Fortune

## Instructions

This project will take you through the process of implementing a simplified version of the game Wheel of Fortune. Here are the rules of our game:

There are `num_human` __human players__ and `num_computer` __computer players__.

* Every player has some amount of money (\$0 at the start of the game)
* Every player has a set of prizes (none at the start of the game)

### the goal

The __goal__ is to __guess a phrase__ within a category. For example:

* Category: Artist & Song
* Phrase: Whitney Houston's I Will Always Love You

Players see the category and an obscured version of the phrase where every alphanumeric character in the phrase starts out as hidden (using underscores: \_):

* Category: Artist & Song
* Phrase: \_\_\_\_\_\_\_ \_\_\_\_\_\_\_'\_ \_ \_\_\_\_ \_\_\_\_\_\_ \_\_\_\_ \_\_\_
* Note that case (capitalization) does not matter

### turn play

During their turn, every player spins the wheel to determine a prize amount and:

#### cash square

If the wheel lands on a cash square, players may do one of three actions:

1. Guess any letter that hasn't been guessed by typing a letter (a-z)

    * Vowels (a, e, i, o, u) cost \$250 to guess and can't be guessed if the player doesn't have enough money. All other letters are "free" to guess.

    * The player can guess any letter that hasn’t been guessed and gets that cash amount for every time that letter appears in the phrase

    * If there is a prize, the user also gets that prize (in addition to any prizes they already had)

    * If the letter does appear in the phrase, the user keeps their turn. Otherwise, it’s the next player’s turn

    * Example: The user lands on \$500 ang guesses 'W' -- There are three Ws in the phrase, so the player wins 1,500.
    
2. Guess the complete phrase by typing a phrase (anything over one character that isn’t 'pass')

    * If they are correct, they win the game

    * If they are incorrect, it is the next player’s turn

3. Pass their turn by entering 'pass'

#### lose a turn

If the wheel lands on "lose a turn", the player loses their turn and the game moves on to the next player

#### bankrupt

If the wheel lands on "bankrupt", the player loses their turn and loses their money but they keep all of the prizes they have won so far.

### winning

The game continues until the entire phrase is revealed or one player guesses the complete phrase.

## Functions / methods

### build suspense with `time.sleep()`

The `time.sleep()` delays execution of the next line of code for s seconds. You’ll find that we can build a little suspense during gameplay with some well-placed delays. The game can also be easier for users to understand if not everything happens instantly.

In [14]:
import time

#time.sleep example:
for x in range(2, 6):
    print('Sleep {} seconds..'.format(x))
    time.sleep(x) # "Sleep" for x seconds
print('Done!')

Sleep 2 seconds..
Sleep 3 seconds..
Sleep 4 seconds..
Sleep 5 seconds..
Done!


### `random.randint()`

The `random` module includes several useful methods for generating and using random numbers, including:

* `random.randint(min, max)` generates a random number between min and max (__inclusive__). It's the same as `randrange(start, stop+1)`.

* `random.choice(l)` selects a random item from the list l (or other iterator, such as strings)

In [15]:
import random

rand_number = random.randint(1, 10)
print('Random number between 1 and 10: {}'.format(rand_number))

letters = [letter for letter in 'ABCDEFGHIJKLMNOPQRSTUVWXYZ']
rand_letter = random.choice(letters)
print('Random letter: {}'.format(rand_letter))

Random number between 1 and 10: 1
Random letter: I


### string methods

String methods we will use for this project:

* `.upper()` converts a string to uppercase (the opposite is `.lower()`)
* `.count(str)` counts how many times the string `str` occurs inside of a larger string

In [16]:
myString = 'Hello, World! 123'

print(myString.upper()) # HELLO, WORLD! 123
print(myString.lower()) # hello, world! 123
print(myString.count('l')) # 3

s = 'python is pythonic'
print(s.count('python')) # 2

HELLO, WORLD! 123
hello, world! 123
3
2


## pre-written methods

* `getNumberBetween(prompt, min, max)`: repeatedly asks the user for a number between `min` and `max` with the prompt `prompt`
* `spinWheel()`: simulates spinning the wheel and returns a dictionary with a random prize
* `getRandomCategoryAndPhrase()`: returns a tuple with a random category and phrase for players to guess
* `obscurePhrase(phrase, guessed)`: returns a tuple with a random category and phrase for players to guess
* `showBoard(category, obscuredPhrase, guessed)`: returns a string representing the current state of the game

In [17]:
import json
import random
import time

LETTERS = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'

# Repeatedly asks the user for a number between min & max (inclusive)
def getNumberBetween(prompt, min, max):
    userinp = input(prompt) # ask the first time

    while True:
        try:
            n = int(userinp) # try casting to an integer
            if n < min:
                errmessage = 'Must be at least {}'.format(min)
            elif n > max:
                errmessage = 'Must be at most {}'.format(max)
            else:
                return n
        except ValueError: # The user didn't enter a number
            errmessage = '{} is not a number.'.format(userinp)

        # If we haven't gotten a number yet, add the error message
        # and ask again
        userinp = input('{}\n{}'.format(errmessage, prompt))

# Spins the wheel of fortune wheel to give a random prize
# Examples:
#    { "type": "cash", "text": "$950", "value": 950, "prize": "A trip to Ann Arbor!" },
#    { "type": "bankrupt", "text": "Bankrupt", "prize": false },
#    { "type": "loseturn", "text": "Lose a turn", "prize": false }
def spinWheel():
    with open("files/wheel.json", 'r') as f:
        wheel = json.loads(f.read())
        return random.choice(wheel)

# Returns a category & phrase (as a tuple) to guess
# Example:
#     ("Artist & Song", "Whitney Houston's I Will Always Love You")
def getRandomCategoryAndPhrase():
    with open("files/phrases.json", 'r') as f:
        phrases = json.loads(f.read())

        category = random.choice(list(phrases.keys()))
        phrase   = random.choice(phrases[category])
        return (category, phrase.upper())

# Given a phrase and a list of guessed letters, returns an obscured version
# Example:
#     guessed: ['L', 'B', 'E', 'R', 'N', 'P', 'K', 'X', 'Z']
#     phrase:  "GLACIER NATIONAL PARK"
#     returns> "_L___ER N____N_L P_RK"
def obscurePhrase(phrase, guessed):
    rv = ''
    for s in phrase:
        if (s in LETTERS) and (s not in guessed):
            rv = rv+'_'
        else:
            rv = rv+s
    return rv

# Returns a string representing the current state of the game
def showBoard(category, obscuredPhrase, guessed):
    return """
Category: {}
Phrase:   {}
Guessed:  {}""".format(category, obscuredPhrase, ', '.join(sorted(guessed)))

category, phrase = getRandomCategoryAndPhrase()

guessed = []
for x in range(random.randint(10, 20)):
    randomLetter = random.choice(LETTERS)
    if randomLetter not in guessed:
        guessed.append(randomLetter)

print("getRandomCategoryAndPhrase()\n -> ('{}', '{}')".format(category, phrase))

print("\n{}\n".format("-"*5))

print("obscurePhrase('{}', [{}])\n -> {}".format(phrase, ', '.join(["'{}'".format(c) for c in guessed]), obscurePhrase(phrase, guessed)))

print("\n{}\n".format("-"*5))

obscured_phrase = obscurePhrase(phrase, guessed)
print("showBoard('{}', '{}', [{}])\n -> {}".format(phrase, obscured_phrase, ','.join(["'{}'".format(c) for c in guessed]), showBoard(phrase, obscured_phrase, guessed)))

print("\n{}\n".format("-"*5))

getRandomCategoryAndPhrase()
 -> ('Author & Title', 'THE GREAT GATSBY BY F. SCOTT FITZGERALD')

-----

obscurePhrase('THE GREAT GATSBY BY F. SCOTT FITZGERALD', ['F', 'L', 'B', 'U', 'G', 'Q', 'H', 'C', 'A', 'Y', 'I', 'V', 'S', 'D'])
 -> _H_ G__A_ GA_SBY BY F. SC___ FI__G__ALD

-----

showBoard('THE GREAT GATSBY BY F. SCOTT FITZGERALD', '_H_ G__A_ GA_SBY BY F. SC___ FI__G__ALD', ['F','L','B','U','G','Q','H','C','A','Y','I','V','S','D'])
 -> 
Category: THE GREAT GATSBY BY F. SCOTT FITZGERALD
Phrase:   _H_ G__A_ GA_SBY BY F. SC___ FI__G__ALD
Guessed:  A, B, C, D, F, G, H, I, L, Q, S, U, V, Y

-----



In [18]:
print(showBoard('THE DEER & HOLLY HUNTER', 'T__ ___R & _O__Y __NT_R', ['I','R','T','F','C','O','Y','N','P','J','V']))



Category: THE DEER & HOLLY HUNTER
Phrase:   T__ ___R & _O__Y __NT_R
Guessed:  C, F, I, J, N, O, P, R, T, V, Y


In [19]:
#change True/False strings to Boolean
print(eval("False"))
print(eval("True"))

False
True


## Part A: WOFPlayer

We’re going to start by defining a class to represent a Wheel of Fortune player, called `WOFPlayer`. Every instance of `WOFPlayer` has three instance variables:

* `.name`: The name of the player (should be passed into the constructor)

* `.prizeMoney`: The amount of prize money for this player (an integer, initialized to `0`)

* `.prizes`: The prizes this player has won so far (a list, initialized to `[]`)

Of these instance variables, only `name` should be passed into the constructor.

It should also have the following methods (note: we will exclude `self` in our descriptions):

* `.addMoney(amt)`: Add `amt` to `self.prizeMoney`

* `.goBankrupt()`: Set `self.prizeMoney` to `0`

* `.addPrize(prize)`: Append `prize` to `self.prizes`

* `.__str__()`: __returns the player’s name and prize money in the following format__:
    * `Steve ($1800)` (for a player with instance variables `.name == 'Steve'` and `prizeMoney == 1800`)

In [20]:
VOWEL_COST = 250
LETTERS = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
VOWELS = 'AEIOU'

In [21]:
# Write the WOFPlayer class definition (part A) here

class WOFPlayer:
    def __init__(self, name):
        self.name = name
        self.prizeMoney = 0
        self.prizes = []
        
    def __str__(self):
        return "{} (${})".format(self.name, self.prizeMoney)
    
    # Add amt to self.prizeMoney
    def addMoney(self,amt):
        self.prizeMoney += amt
        return None
    
    # Set self.prizeMoney to 0
    def goBankrupt(self):
        self.prizeMoney = 0
        return None
    
    # Append prize to self.prizes
    def addPrize(self,prize):
        self.prizes.append(prize)
        return None

In [24]:
# TDD
from test import testEqual
wof = WOFPlayer('Adina')
wof.prizeMoney = 100

testEqual(wof.name, 'Adina')
testEqual(wof.prizeMoney, 100)
testEqual(wof.prizes, [])

print(wof)

wof.addMoney(100)
testEqual(wof.prizeMoney, 200)

wof.goBankrupt()
testEqual(wof.prizeMoney, 0)

wof.addPrize('boat')
testEqual(wof.prizes, ['boat'])

	Pass
	Pass
	Pass
Adina ($100)
	Pass
	Pass
	Pass


True

## Part B: WOFHumanPlayer

Next, we’re going to define a class named `WOFHumanPlayer`, which should inherit from `WOFPlayer` (part A). This class is going to represent a human player. In addition to having all of the instance variables and methods that `WOFPlayer` has, `WOFHumanPlayer` should have an additional method:

### getMove()

`.getMove(category, obscuredPhrase, guessed)`: __Prompts__ the user __to enter a move__ and __returns whatever string they entered__.

`.getMove()`’s prompt should be:

```
{name} has ${prizeMoney}

Category: {category}
Phrase:  {obscured_phrase}
Guessed: {guessed}

Guess a letter, phrase, or type 'exit' or 'pass':
```

### example:

```
Steve has $200

Category: Places
Phrase: _L___ER N____N_L P_RK
Guessed: B, E, K, L, N, P, R, X, Z

Guess a letter, phrase, or type 'exit' or 'pass':
```

The user can then enter:

* `'exit'` to exit the game
* `'pass'` to skip their turn
* a single character to guess that letter
* a complete phrase (a multi-character phrase other than `'exit'` or `'pass'`) to guess that phrase

__Note:__ `.getMove()` does __not__ need to enforce anything about the user’s input; that will be done via the game logic that we define in the next ActiveCode window.

In [25]:
# Write the WOFHumanPlayer class definition (part B) here

class WOFHumanPlayer(WOFPlayer):
    def getMove(self, category, obscuredPhrase, guessed):
        move = input("""
{} has ${}
{}\n
Guess a letter, phrase, or type "exit" or "pass":\n
""".format(self.name, self.prizeMoney, showBoard(category, obscuredPhrase, guessed))
        )
        return move

In [26]:
#TDD
wofh = WOFHumanPlayer('Lydia')
wofh.getMove('Places', '_L___ER N____N_L P_RK', ['B', 'E', 'K', 'L', 'N', 'P', 'R', 'X', 'Z'])



Lydia has $0

Category: Places
Phrase:   _L___ER N____N_L P_RK
Guessed:  B, E, K, L, N, P, R, X, Z

Guess a letter, phrase, or type "exit" or "pass":

l


'l'

## Part C: WOFComputerPlayer

Finally, we’re going to define a class named `WOFComputerPlayer`, which should inherit from `WOFPlayer` (part A). This class is going to represent a computer player.

Every computer player will have a `difficulty` instance variable. Players with a higher `difficulty` generally play “better”. There are many ways to implement this. We’ll do the following:

* If there aren’t any possible letters to choose (for example: if the last character is a vowel but this player doesn’t have enough to guess a vowel), we’ll `'pass'`
* __Otherwise, semi-randomly decide whether to make a “good” move or a “bad” move on a given turn (a higher difficulty should make it more likely for the player to make a “good” move)__
    * To make a “bad” move, we’ll randomly decide on a possible letter.
    * To make a “good” move, we’ll choose a letter according to their overall frequency in the English language.

In addition to having all of the instance variables and methods that `WOFPlayer` has, `WOFComputerPlayer` should have:

### class variable

`.SORTED_FREQUENCIES`: Should be set to `'ZQXJKVBPYGFWMUCLDRHSNIOATE'`, which is a list of English characters sorted from least frequent ('Z') to most frequent ('E'). We’ll use this when trying to make a “good” move.

### additional instance variable

`.difficulty`: The level of difficulty for this computer (should be passed as the __second argument__ into the __constructor__ after `.name`)

### methods

* `.smartCoinFlip()`: This method will help us decide semi-randomly whether to make a “good” or “bad” move. A higher difficulty should make us more likely to make a “good” move. Implement this by choosing a random number between `1` and `10` using `random.randint(1, 10)` (see above) and returning `False` if that random number is greater than `self.difficulty`. If the random number is less than or equal to `self.difficulty`, return `True`.

* `.getPossibleLetters(guessed)`: This method should __return a list of letters that can be guessed__.
    * These should be characters that are in `LETTERS` (`'ABCDEFGHIJKLMNOPQRSTUVWXYZ'`) but __not__ in the `guessed` parameter.

    * Additionally, if this player doesn’t have enough prize money to guess a vowel (variable `VOWEL_COST` set to `250`), then vowels (variable `VOWELS` set to `'AEIOU'`) should __not__ be included

* `.getMove(category, obscuredPhrase, guessed)`: __Returns a valid move__ (__doesn't prompt__ of course).
    * Use the `.getPossibleLetters(guessed)` method described above.

    * If there aren’t any letters that can be guessed (this can happen if the only letters left to guess are vowels and the player doesn’t have enough for vowels), return `'pass'`.

    * Use the `.smartCoinFlip()` method to __decide whether to make a “good” or a “bad” move__.
        * If making a “good” move (`.smartCoinFlip()` returns `True`), then return the most frequent (highest index in `.SORTED_FREQUENCIES`) possible character

        * If making a “bad” move (`.smartCoinFlip()` returns `False`), then return a random character from the set of possible characters (use `random.choice()`)

In [72]:
# Write the WOFComputerPlayer class definition (part C) here

class WOFComputerPlayer(WOFPlayer):
    SORTED_FREQUENCIES = 'ZQXJKVBPYGFWMUCLDRHSNIOATE'
    
    def __init__(self, name, difficulty):
        super().__init__(name)
        self.difficulty = difficulty
    
    # decide semi-randomly whether to make a “good” or “bad” move.
    def smartCoinFlip(self):
        # choose a random number between 1 & 10
        rand = random.randint(1,10)
        # if number > difficulty return False, else return True
        if rand > self.difficulty:
            return False
        return True
    
    # return a list of letters that can be guessed
    def getPossibleLetters(self, guessed):
        # remove guessed letters from LETTERS
        possible = [l for l in list(LETTERS) if l not in guessed]
        # remove vowels if not enough prizeMoney < VOWEL_COST remove VOWELS
        if self.prizeMoney < VOWEL_COST:
            possible = [l for l in possible if l not in VOWELS]
        return possible
    
    # return a valid move
    def getMove(self, category, obscuredPhrase, guessed):
        if self.smartCoinFlip():
            # SMART computer takes last letter from possible letters
            # if no possible letters, it passes
            possible = [l for l in self.SORTED_FREQUENCIES if l not in guessed]
            if self.prizeMoney < VOWEL_COST:
                possible = [l for l in possible if l not in VOWELS]
            return possible[-1] if possible != [] else 'pass'
        # DUMB computer chooses a letter from available letters
        # if no possible letters, it passes
        possible = self.getPossibleLetters(guessed)
        return random.choice(possible) if possible != [] else 'pass'

In [73]:
#TDD

wofc = WOFComputerPlayer('Jonathan', 6)

# TEST INIT
print('\ntesting init:')
testEqual(wofc.name, 'Jonathan')
testEqual(wofc.prizeMoney, 0)
testEqual(wofc.prizes, [])
testEqual(wofc.SORTED_FREQUENCIES, 'ZQXJKVBPYGFWMUCLDRHSNIOATE')
testEqual(wofc.difficulty, 6)
print('---')
print(wofc)


# TEST METHOD: smartCoinFlip
print('\ntesting smartCoinFlip:')
testEqual(wofc.smartCoinFlip(), False or True)

print('---')

wofc.difficulty = -1
for n in range(1,11):
    results = testEqual(wofc.smartCoinFlip(), False)
    if results == False:
        break
        
print('---')

wofc.difficulty = 11
for n in range(1,11):
    results = testEqual(wofc.smartCoinFlip(), True)
    if results == False:
        break


# TEST METHOD: getPossibleLetters
print('\ntesting getPossibleLetters:')

print('\n* with enough money to buy a vowel:')
wofc.prizeMoney = 250
g = [] # no letters guessed
testEqual(len(wofc.getPossibleLetters(g)), 26)
g = list(LETTERS) # all letters guessed
testEqual(len(wofc.getPossibleLetters(g)), 0)
g = ['B', 'E', 'K', 'L', 'N', 'P', 'R', 'X', 'Z'] # some letters guessed
testEqual(len(wofc.getPossibleLetters(g)), 26 - len(g))

print('\n* without enough money to buy a vowel:')
wofc.prizeMoney = 5
g = [] # no letters guessed
testEqual(len(wofc.getPossibleLetters(g)), 26 - len(VOWELS))
g = list(LETTERS) # all letters guessed
testEqual(len(wofc.getPossibleLetters(g)), 0)
g = ['B', 'E', 'K', 'L', 'N', 'P', 'R', 'X', 'Z'] # some letters guessed
v = [l for l in VOWELS if l not in g]
testEqual(len(wofc.getPossibleLetters(g)), 26 - len(g) - len(v))


# TEST METHOD: getMove
print('\ntesting getMove:')
catg = 'Places'
phrase = '_______ ________ ____'
g = []
wofc.prizeMoney = 250
move = wofc.getMove(catg, phrase, g)
testEqual(move in LETTERS, True)

wofc.prizeMoney = 5
move = wofc.getMove(catg, phrase, g)
testEqual(move not in VOWELS, 1)

print('-----')

g = [l for l in LETTERS if l not in VOWELS]
wofc.prizeMoney = 250
move = wofc.getMove(catg, phrase, g)
print(move)
testEqual(move in VOWELS, True)

wofc.prizeMoney = 5
move = wofc.getMove(catg, phrase, g)
print(move)
testEqual(move, 'pass')

print('-----')

wofc.prizeMoney = 250
phrase = '_L___ER N____N_L P_RK'
g = ['B', 'E', 'K', 'L', 'N', 'P', 'R', 'X', 'Z']

print('dumb computer:')
wofc.difficulty = -1
available = [l for l in list(LETTERS) if l not in g]
for n in range(1,11):
    move = wofc.getMove(catg, phrase, g)
    results = testEqual(move in available, True)
    if results == False:
        break

print('-----')
print('smart computer:')
wofc.difficulty = 11
available = [l for l in list(wofc.SORTED_FREQUENCIES) if l not in g]
for n in range(1,11):
    move = wofc.getMove(catg, phrase, g)
    results = testEqual(move, available[-1])
    if results == False:
        break


testing init:
	Pass
	Pass
	Pass
	Pass
	Pass
---
Jonathan ($0)

testing smartCoinFlip:
	Pass
---
	Pass
	Pass
	Pass
	Pass
	Pass
	Pass
	Pass
	Pass
	Pass
	Pass
---
	Pass
	Pass
	Pass
	Pass
	Pass
	Pass
	Pass
	Pass
	Pass
	Pass

testing getPossibleLetters:

* with enough money to buy a vowel:
	Pass
	Pass
	Pass

* without enough money to buy a vowel:
	Pass
	Pass
	Pass

testing getMove:
	Pass
	Pass
-----
E
	Pass
pass
	Pass
-----
dumb computer:
	Pass
	Pass
	Pass
	Pass
	Pass
	Pass
	Pass
	Pass
	Pass
	Pass
-----
smart computer:
	Pass
	Pass
	Pass
	Pass
	Pass
	Pass
	Pass
	Pass
	Pass
	Pass
